<a href="https://colab.research.google.com/github/Wuuuuu05/Mask/blob/main/Mask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json
url = https://raw.githubusercontent.com/kiang/pharmacies/master/json/points.json
response = request.get(url)
d = response.text
data = json.loads(d)
print(data)

med_count = {}
for d in data['features']:
    conunty = d['properties']['county']
    if conunty not in med_count:
      med_count[conunty] = 1
    else:
      med_count[conunty] += 1
print(med_count)